In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

OK, before any thing, lets make sure you understand something... then move on:

In [ ]:
g=np.arange(0,10,2) # Lets create a,n array
g

In [ ]:
g=g.reshape(1,5,1) # reshape to higher dimentions
g

In [ ]:
b=np.array([22])  # Single array number
b.shape

In [ ]:
#Now lets add b to the end of g and get rid of the first element in g

g=np.append(g[:,1:,:],[[b]],axis=1)
g

OK, Now that you understood this, Continue....

In [ ]:
df=pd.read_csv('/content/Alcohol_Sales.csv',index_col='DATE',parse_dates=True)

In [ ]:
df.index.freq='MS'

In [ ]:
df.head()

In [ ]:
df.columns=['Sales']

In [ ]:
df.plot()

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

In [ ]:
results=seasonal_decompose(df['Sales'])

In [ ]:
results.plot();

In [ ]:
results.seasonal.plot(figsize=(12,8))

# Our goal is to forecast 1 year into future

In [ ]:
# So how long is our data?

In [ ]:
df.shape

In [ ]:
#Thats 325 months, and we want 12 months

In [ ]:
325-12


In [ ]:
train=df.iloc[:313]
test=df.iloc[313:]

In [ ]:
test.shape

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler=MinMaxScaler()

In [ ]:
scaler.fit(train)   # Basically its locating the max value in the proposed data

In [ ]:
scaled_train=scaler.transform(train)
scaled_test=scaler.transform(test)

#Now lets go with the time series generator from Tenmsorflow

In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

In [ ]:
n_input=3
n_feature=1

In [ ]:
generator=TimeseriesGenerator(scaled_train,scaled_train,length=n_input,batch_size=1)#  We are saying the source of data and its lbel are from scaled train

In [ ]:
generator[0]

In [ ]:
# Do you agree, we had a tuple here?

In [ ]:
X,y=generator[0]

In [ ]:
X

In [ ]:
y

# Now regarding our data

In [ ]:
n_input=12
n_feature=1

train_generator=TimeseriesGenerator(scaled_train,scaled_train,length=n_input,batch_size=1)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

In [ ]:
model=Sequential()  # Sequential way
model.add(LSTM(180,activation='relu',input_shape=(n_input,n_feature)))
model.add(Dense(1))

In [ ]:
from tensorflow.keras.models import Model  # Functional Way, as I like it
from tensorflow.keras.layers import Input, LSTM, Dense


# Functional API
input_layer = Input(shape=(n_input, n_feature), name="Input_Layer")  # Input layer
x = LSTM(180, activation='relu', name="LSTM_Layer")(input_layer)     # LSTM layer
output_layer = Dense(1, name="Output_Layer")(x)                      # Output layer

# Create the model
model = Model(inputs=input_layer, outputs=output_layer, name="Functional_LSTM_Model")

In [ ]:
# Giving the model life, Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mse')

In [ ]:
model.fit(train_generator,epochs=25)

In [ ]:
model.history.history.plot()

In [ ]:
#Lets see how it did?

In [ ]:
# We see 12 steps and predict the 13 motnhs

In [ ]:
first_real_batch=scaled_train[-12:]

In [ ]:
first_real_batch

In [ ]:
first_real_batch.shape

In [ ]:
first_real_batch=first_real_batch.reshape(1,12,1)

In [ ]:
first_real_batch.shape

Now, lets predict

In [ ]:
model.predict(first_real_batch)

In [ ]:
first_eval_batch.shape

In [ ]:
test_preds=[]  # storing our prediction

first_eval_batch=scaled_train[-n_input:]  #  [-12:]

current_batch=first_real_batch

for i in range(len(test)):  #  This len(test)  can be your choice as into steps in future

  spot_pred=model.predict(current_batch)[0]

  test_preds.append(spot_pred)

  current_batch=np.append(current_batch[:,1:,:],[[spot_pred]],axis=1)




In [ ]:
preds=scaler.inverse_transform(test_preds)

In [ ]:
test

In [ ]:
test.loc[:, 'LSTM_preds'] = np.round(preds,1)

In [ ]:
test

In [ ]:
test.plot()

In [ ]:
plt.figure(figsize=(20, 8))

# Plot the training data
plt.plot(train.index, train, label="Training Data", color="blue")

# Plot the real values in the test set
plt.plot(test.index, test['Sales'], label="Real Values", color="green")

# Plot the LSTM predictions
plt.plot(test.index, test['LSTM_preds'], label="LSTM Predictions", color="red")

# Add labels, title, and legend
plt.xlabel("Date", fontsize=14)
plt.ylabel("Wine Price", fontsize=14)
plt.title("Time Series: Wine Prices and LSTM Predictions", fontsize=16)
plt.legend(fontsize=12)

In [ ]:
test.index

In [ ]:
train_filtered = train['2012-01-01':'2018-01-01']
test_filtered = test['2018-02-01':'2019-01-01']

In [ ]:
plt.figure(figsize=(20, 8))
plt.plot(train_filtered, label="Training Data")
plt.plot(test_filtered['Sales'], label="Real Values")
plt.plot(test_filtered['LSTM_preds'], label="LSTM Predictions")
plt.xlabel("Date")
plt.ylabel("Wine Price")
plt.title("Filtered Time Series: Wine Prices and LSTM Predictions")
plt.legend()
plt.grid(True)
plt.show()